## **Preprocessing dát o VŠ učiteľoch**

In [1]:
import pandas as pd

### Functions definitions:

In [2]:
def fix_table(df, university_type=None):
    """
    Fill the university names. Remove summary rows. Eventually add type of university.
    """
    # backfill / bfill: replaces NaN values with the next non-NaN value that appears in the column
    df['univerzita'] = df['univerzita'].fillna(method='backfill')
    # remove summary rows = rows without specified Faculty
    df = df.dropna(axis=0, subset='fakulta').reset_index(drop=True)
    if university_type:
        df['typ'] = university_type
    return df

In [3]:
def load_summary_table(file_path, sheet_name_fulltime, sheet_name_parttime, university_type, year):
    """
    Loads summary data about universities and their numbers of teachers into a dataframe.
    """
    df_fulltime = fix_table(pd.read_excel(file_path, sheet_name=sheet_name_fulltime, header=5, usecols="A:D"
                                         ).rename(columns={'Unnamed: 0': 'univerzita', 'Unnamed: 1': 'fakulta', 
                                                           'Unnamed: 2': 'spolu full-time', 'Unnamed: 3': 'ženy full-time'}),
                            university_type=university_type)
    df_parttime = fix_table(pd.read_excel(file_path, sheet_name=sheet_name_parttime, header=5, usecols="A:D"
                                         ).rename(columns={'Unnamed: 0': 'univerzita', 'Unnamed: 1': 'fakulta', 
                                                           'Unnamed: 2': 'spolu part-time', 'Unnamed: 3': 'ženy part-time'}),
                            university_type=university_type)
    merged = pd.merge(df_fulltime, df_parttime, how="outer", on=['univerzita', 'fakulta', 'typ']
                     ).fillna(0).astype({'spolu full-time': int, 'spolu part-time': int, 'ženy full-time': int, 'ženy part-time': int})
    merged['rok'] = year
                                         
    return merged[['univerzita', 'fakulta', 'typ', 'rok', 'spolu full-time', 'spolu part-time', 'ženy full-time', 'ženy part-time']]

In [4]:
def add_type_of_teacher(df, type_of_teacher):
    """
    Add column for type_of_teacher.
    """
    df['typ učiteľa'] = type_of_teacher
    return df

In [5]:
def create_degrees_table(file_path, sheet_name, commitment, year, university_type):
    """
    Loads data about teachers and their degrees into a dataframe.
    """
    df_prof = add_type_of_teacher(fix_table(pd.read_excel(file_path, sheet_name=sheet_name, usecols="A:B, E:G", header=5
                                                         ).rename(columns={'Unnamed: 0': 'univerzita', 'Unnamed: 1': 'fakulta', 
                                                                  'Unnamed: 4': 'spolu', 'Unnamed: 5': 'DrSc.', 'PhD.': 'CSc., PhD.'}),
                                           university_type=university_type),
                                  type_of_teacher='profesor')
    df_doc = add_type_of_teacher(fix_table(pd.read_excel(file_path, sheet_name=sheet_name, usecols="A:B, H:J", header=5
                                                        ).rename(columns={'Unnamed: 0': 'univerzita', 'Unnamed: 1': 'fakulta', 
                                                                 'Unnamed: 7': 'spolu', 'Unnamed: 8': 'DrSc.', 'PhD..1': 'CSc., PhD.'}),
                                          university_type=university_type),
                                 type_of_teacher='docent')
    df_odb = add_type_of_teacher(fix_table(pd.read_excel(file_path, sheet_name=sheet_name, usecols="A:B, K:L", header=5
                                                        ).rename(columns={'Unnamed: 0': 'univerzita', 'Unnamed: 1': 'fakulta', 
                                                                 'Unnamed: 10': 'spolu', 'PhD..2': 'CSc., PhD.'}),
                                          university_type=university_type),
                                 type_of_teacher='odborný asistent')
    df_asis = add_type_of_teacher(fix_table(pd.read_excel(file_path, sheet_name=sheet_name, usecols="A:B, M", header=5
                                                        ).rename(columns={'Unnamed: 0': 'univerzita', 'Unnamed: 1': 'fakulta', 
                                                                 'Unnamed: 12': 'spolu'}),
                                           university_type=university_type),
                                 type_of_teacher='asistent')
    df_lekt = add_type_of_teacher(fix_table(pd.read_excel(file_path, sheet_name=sheet_name, usecols="A:B, N", header=5
                                                        ).rename(columns={'Unnamed: 0': 'univerzita', 'Unnamed: 1': 'fakulta', 
                                                                 'Unnamed: 13': 'spolu'}),
                                           university_type=university_type),
                                 type_of_teacher='lektor')
    df_host = add_type_of_teacher(fix_table(pd.read_excel(file_path, sheet_name=sheet_name, usecols="A:B, O", header=5
                                                        ).rename(columns={'Unnamed: 0': 'univerzita', 'Unnamed: 1': 'fakulta', 
                                                                 'Unnamed: 14': 'spolu'}),
                                           university_type=university_type),
                                 type_of_teacher='hosťujúci profesor')
    concatenated = pd.concat([df_prof, df_doc, df_odb, df_asis, df_lekt, df_host], axis=0, join='outer'
                            ).fillna(0).astype({'DrSc.': int, 'CSc., PhD.': int, 'spolu': int})
    concatenated['iný/žiadny titul'] = concatenated['spolu'] - concatenated['DrSc.'] - concatenated['CSc., PhD.']
    concatenated['úväzok'] = commitment
    concatenated['rok'] = year
    
    return concatenated[['univerzita', 'fakulta', 'typ', 'rok', 'typ učiteľa', 'úväzok', 'DrSc.', 'CSc., PhD.', 'iný/žiadny titul', 'spolu']].reset_index(drop=True)
    

In [6]:
def load_all(file_path, year):
    """
    Loads all data about teachers from a certain file_path for a certain year and stores them into 2 dataframes: summary and degrees.
    """
    public = load_summary_table(file_path, sheet_name_fulltime='Tab16v', sheet_name_parttime='Tab17v', university_type='verejná', year=year)
    private = load_summary_table(file_path, sheet_name_fulltime='Tab16s', sheet_name_parttime='Tab17s', university_type='súkromná', year=year)
    state = load_summary_table(file_path, sheet_name_fulltime='Tab16š', sheet_name_parttime='Tab17š', university_type='štátna', year=year)
    summary = pd.concat([public, private, state]).reset_index(drop=True)
    
    degrees_fulltime_v = create_degrees_table(file_path=file_path, sheet_name='Tab16v', commitment='full-time', year=year, university_type='verejná')
    degrees_parttime_v = create_degrees_table(file_path=file_path, sheet_name='Tab17v', commitment='part-time', year=year, university_type='verejná')
    degrees_fulltime_s = create_degrees_table(file_path=file_path, sheet_name='Tab16s', commitment='full-time', year=year, university_type='súkromná')
    degrees_parttime_s = create_degrees_table(file_path=file_path, sheet_name='Tab17s', commitment='part-time', year=year, university_type='súkromná')
    degrees_fulltime_st = create_degrees_table(file_path=file_path, sheet_name='Tab16š', commitment='full-time', year=year, university_type='štátna')
    degrees_parttime_st = create_degrees_table(file_path=file_path, sheet_name='Tab17š', commitment='part-time', year=year, university_type='štátna')
    degrees = pd.concat([degrees_fulltime_v, degrees_parttime_v, degrees_fulltime_s, 
                         degrees_parttime_s, degrees_fulltime_st, degrees_parttime_st]).reset_index(drop=True)
    
    return summary, degrees

### Load the data:

#### - rok 2022:

In [7]:
public_22 = load_summary_table('../data/cvtisr/vs2022/vs2022.xls', sheet_name_fulltime='Tab16v', sheet_name_parttime='Tab17v', university_type='verejná', year=2022)
display(public_22)

,univerzita,fakulta,typ,rok,spolu full-time,spolu part-time,ženy full-time,ženy part-time
0,UNIVERZITA KOMENSKÉHO,Rímskokat.bohosl.fak.UK,verejná,2022,21,19,2,3
1,UNIVERZITA KOMENSKÉHO,Evanj. bohosl. fak. UK,verejná,2022,14,2,2,2
2,UNIVERZITA KOMENSKÉHO,Fa matem.fyz. a inf. UK,verejná,2022,211,21,63,5
3,UNIVERZITA KOMENSKÉHO,Prírodovedecká fak. UK,verejná,2022,319,48,139,20
4,UNIVERZITA KOMENSKÉHO,Fakulta managementu UK,verejná,2022,69,2,39,1
...,...,...,...,...,...,...,...,...
112,VŠ MÚZICKÝCH UMENÍ,VŠMU - rektorát,verejná,2022,3,2,3,2
113,VŠ VÝTVARNÝCH UMENÍ,Fa výtvar. umení VŠVU,verejná,2022,118,12,40,4
114,AKADÉMIA UMENÍ,Fa dramat. umení AU,verejná,2022,35,2,10,1
115,AKADÉMIA UMENÍ,Fa múzických umení AU,verejná,2022,66,8,23,4


In [8]:
private_22 = load_summary_table('../data/cvtisr/vs2022/vs2022.xls', sheet_name_fulltime='Tab16s', sheet_name_parttime='Tab17s', university_type='súkromná', year=2022)
display(private_22)

,univerzita,fakulta,typ,rok,spolu full-time,spolu part-time,ženy full-time,ženy part-time
0,BRATISLAVSKÁ MEDZ.ŠKOLA,Fa liberálnych štúdií,súkromná,2022,6,9,3,1
1,VŠ DANUBIUS,Fa verejnej politiky,súkromná,2022,6,3,3,0
2,VŠ DANUBIUS,Fa práva J.Jesenského,súkromná,2022,9,4,3,1
3,VŠ DANUBIUS,Fa sociálnych štúdií,súkromná,2022,9,0,3,0
4,PANEURÓPSKA VŠ,Fa informatiky PEVŠ,súkromná,2022,8,0,0,0
5,PANEURÓPSKA VŠ,Fa ekon. a podnikania,súkromná,2022,10,3,5,0
6,PANEURÓPSKA VŠ,Fa práva PEVŠ,súkromná,2022,24,6,12,1
7,PANEURÓPSKA VŠ,Fa masmédií PEVŠ,súkromná,2022,11,4,4,0
8,PANEURÓPSKA VŠ,Fa psychológie PEVŠ,súkromná,2022,15,0,12,0
9,VŠZaSP sv.ALŽBETY,VŠZaSP - rektorát,súkromná,2022,250,244,147,152


In [9]:
state_22 = load_summary_table('../data/cvtisr/vs2022/vs2022.xls', sheet_name_fulltime='Tab16š', sheet_name_parttime='Tab17š', university_type='štátna', year=2022)
display(state_22)

,univerzita,fakulta,typ,rok,spolu full-time,spolu part-time,ženy full-time,ženy part-time
0,SLOV.ZDRAVOTNÍCKA UNIV.,SZU - rektorát,štátna,2022,1,0,0,0
1,SLOV.ZDRAVOTNÍCKA UNIV.,"Fa ošetr.,zdrav.odb.štú",štátna,2022,26,47,18,31
2,SLOV.ZDRAVOTNÍCKA UNIV.,Lekárska fakulta,štátna,2022,86,245,40,119
3,SLOV.ZDRAVOTNÍCKA UNIV.,Fa verejného zdravotn.,štátna,2022,16,7,11,5
4,SLOV.ZDRAVOTNÍCKA UNIV.,Fa zdravotníctva,štátna,2022,24,64,15,39
5,AKADÉMIA OZBROJ.SÍL,Vojenská fakulta AOS,štátna,2022,84,0,16,0
6,AKADÉMIA POLIC.ZBORU,Fa Polic.zboru (fikt.),štátna,2022,92,10,44,2


In [10]:
summary = pd.concat([public_22, private_22, state_22]).reset_index(drop=True)
display(summary)

,univerzita,fakulta,typ,rok,spolu full-time,spolu part-time,ženy full-time,ženy part-time
0,UNIVERZITA KOMENSKÉHO,Rímskokat.bohosl.fak.UK,verejná,2022,21,19,2,3
1,UNIVERZITA KOMENSKÉHO,Evanj. bohosl. fak. UK,verejná,2022,14,2,2,2
2,UNIVERZITA KOMENSKÉHO,Fa matem.fyz. a inf. UK,verejná,2022,211,21,63,5
3,UNIVERZITA KOMENSKÉHO,Prírodovedecká fak. UK,verejná,2022,319,48,139,20
4,UNIVERZITA KOMENSKÉHO,Fakulta managementu UK,verejná,2022,69,2,39,1
...,...,...,...,...,...,...,...,...
136,SLOV.ZDRAVOTNÍCKA UNIV.,Lekárska fakulta,štátna,2022,86,245,40,119
137,SLOV.ZDRAVOTNÍCKA UNIV.,Fa verejného zdravotn.,štátna,2022,16,7,11,5
138,SLOV.ZDRAVOTNÍCKA UNIV.,Fa zdravotníctva,štátna,2022,24,64,15,39
139,AKADÉMIA OZBROJ.SÍL,Vojenská fakulta AOS,štátna,2022,84,0,16,0


In [11]:
degrees_fulltime_v = create_degrees_table(file_path='../data/cvtisr/vs2022/vs2022.xls', sheet_name='Tab16v', commitment='full-time', year=2022, university_type='verejná')
degrees_parttime_v = create_degrees_table(file_path='../data/cvtisr/vs2022/vs2022.xls', sheet_name='Tab17v', commitment='part-time', year=2022, university_type='verejná')

degrees_fulltime_s = create_degrees_table(file_path='../data/cvtisr/vs2022/vs2022.xls', sheet_name='Tab16s', commitment='full-time', year=2022, university_type='súkromná')
degrees_parttime_s = create_degrees_table(file_path='../data/cvtisr/vs2022/vs2022.xls', sheet_name='Tab17s', commitment='part-time', year=2022, university_type='súkromná')

degrees_fulltime_st = create_degrees_table(file_path='../data/cvtisr/vs2022/vs2022.xls', sheet_name='Tab16š', commitment='full-time', year=2022, university_type='štátna')
degrees_parttime_st = create_degrees_table(file_path='../data/cvtisr/vs2022/vs2022.xls', sheet_name='Tab17š', commitment='part-time', year=2022, university_type='štátna')

degrees = pd.concat([degrees_fulltime_v, degrees_parttime_v, degrees_fulltime_s, 
                        degrees_parttime_s, degrees_fulltime_st, degrees_parttime_st]).reset_index(drop=True)
display(degrees)

,univerzita,fakulta,typ,rok,typ učiteľa,úväzok,DrSc.,"CSc., PhD.",iný/žiadny titul,spolu
0,UNIVERZITA KOMENSKÉHO,Rímskokat.bohosl.fak.UK,verejná,2022,profesor,full-time,0,6,0,6
1,UNIVERZITA KOMENSKÉHO,Evanj. bohosl. fak. UK,verejná,2022,profesor,full-time,0,4,0,4
2,UNIVERZITA KOMENSKÉHO,Fa matem.fyz. a inf. UK,verejná,2022,profesor,full-time,17,26,0,43
3,UNIVERZITA KOMENSKÉHO,Prírodovedecká fak. UK,verejná,2022,profesor,full-time,17,32,0,49
4,UNIVERZITA KOMENSKÉHO,Fakulta managementu UK,verejná,2022,profesor,full-time,0,15,0,15
...,...,...,...,...,...,...,...,...,...,...
1537,SLOV.ZDRAVOTNÍCKA UNIV.,"Fa ošetr.,zdrav.odb.štú",štátna,2022,hosťujúci profesor,part-time,0,0,0,0
1538,SLOV.ZDRAVOTNÍCKA UNIV.,Lekárska fakulta,štátna,2022,hosťujúci profesor,part-time,0,0,0,0
1539,SLOV.ZDRAVOTNÍCKA UNIV.,Fa verejného zdravotn.,štátna,2022,hosťujúci profesor,part-time,0,0,0,0
1540,SLOV.ZDRAVOTNÍCKA UNIV.,Fa zdravotníctva,štátna,2022,hosťujúci profesor,part-time,0,0,0,0


In [12]:
# summary.to_csv('../data/cvtisr/preprocessed/summary22.csv', sep=';', index=False)
# degrees.to_csv('../data/cvtisr/preprocessed/degrees22.csv', sep=';', index=False)

#### - všetky roky 2009-2022: načítanie dát, predspracovanie a uloženie do csv

In [13]:
whole_summary, whole_degrees = pd.DataFrame(), pd.DataFrame()
for year in range(2009, 2023):
    path = f"../data/cvtisr/vs{year}/vs{year}.xls"
    year = path[-8:-4]
    print(f"Preprocessing data from year {year}")
    summary_year, degrees_year = load_all(file_path=path, year=year)
    
    whole_summary = pd.concat([whole_summary, summary_year]).reset_index(drop=True)
    whole_degrees = pd.concat([whole_degrees, degrees_year]).reset_index(drop=True)
    
whole_summary.to_csv(f'../preprocessing/data_cvtisr_preprocessed/cvtisr_teachers_summary.csv', sep=';', index=False)
whole_degrees.to_csv(f'../preprocessing/data_cvtisr_preprocessed/cvtisr_teachers_data_degrees.csv', sep=';', index=False)
    

Preprocessing data from year 2009
Preprocessing data from year 2010
Preprocessing data from year 2011
Preprocessing data from year 2012
Preprocessing data from year 2013
Preprocessing data from year 2014
Preprocessing data from year 2015
Preprocessing data from year 2016
Preprocessing data from year 2017
Preprocessing data from year 2018
Preprocessing data from year 2019
Preprocessing data from year 2020
Preprocessing data from year 2021
Preprocessing data from year 2022


## **Preprocessing dát o VŠ absolventoch**

In [29]:
def load_graduates(file_path, sheet_name, university_type, year):
    """
    Loads data about graduates from a given sheet into a dataframe.
    """
    df = fix_table(pd.read_excel(file_path, sheet_name=sheet_name, usecols="A:B, C, E, G, I, K, M, O", header=5
                                ).rename(columns={'Unnamed: 0': 'univerzita', 'Unnamed: 1': 'fakulta'}),
                   university_type=university_type)
    df['abs 1.-2.stupeň denní'] = (df['spolu'] + df['spolu.1']).astype(int)
    df['abs 1.-2.stupeň externí'] = df['spolu.2'].astype(int)
    df['abs 3.stupeň denní'] = (df['spolu.3'] + df['spolu.4']).astype(int)
    df['abs 3.stupeň externí'] = df['spolu.5'].astype(int)
    df['abs 3.stupeň externých vzdelávacích inštitúcií'] = df['spolu.6'].astype(int)
    df['rok'] = year
    return df[['univerzita', 'fakulta', 'typ', 'rok', 'abs 1.-2.stupeň denní', 'abs 1.-2.stupeň externí',
             'abs 3.stupeň denní', 'abs 3.stupeň externí', 'abs 3.stupeň externých vzdelávacích inštitúcií']]
    

In [30]:
def load_all_graduates(file_path, year):
    """
    Loads data about graduates from a fiven file into a dataframe.
    """
    public = load_graduates(file_path, sheet_name='Tab1v', university_type='verejná', year=year)
    private = load_graduates(file_path, sheet_name='Tab1s', university_type='súkromná', year=year)
    state = load_graduates(file_path, sheet_name='Tab1š', university_type='štátna', year=year)
    all_data = pd.concat([public, private, state]).reset_index(drop=True)
    
    return all_data

In [31]:
load_all_graduates("../data/cvtisr/vs2021/abvs_2021.xls", year=2021)

,univerzita,fakulta,typ,rok,abs 1.-2.stupeň denní,abs 1.-2.stupeň externí,abs 3.stupeň denní,abs 3.stupeň externí,abs 3.stupeň externých vzdelávacích inštitúcií
0,UNIVERZITA KOMENSKÉHO,Rímskokat.bohosl.fak.UK,verejná,2021,13,0,0,1,0
1,UNIVERZITA KOMENSKÉHO,Evanj. bohosl. fak. UK,verejná,2021,3,0,1,0,0
2,UNIVERZITA KOMENSKÉHO,Fa matem.fyz. a inf. UK,verejná,2021,307,0,26,1,5
3,UNIVERZITA KOMENSKÉHO,Prírodovedecká fak. UK,verejná,2021,561,2,70,6,26
4,UNIVERZITA KOMENSKÉHO,Fakulta managementu UK,verejná,2021,421,227,7,21,0
...,...,...,...,...,...,...,...,...,...
130,SLOV.ZDRAVOTNÍCKA UNIV.,Lekárska fakulta,štátna,2021,106,0,0,5,0
131,SLOV.ZDRAVOTNÍCKA UNIV.,Fa verejného zdravotn.,štátna,2021,11,8,1,16,0
132,SLOV.ZDRAVOTNÍCKA UNIV.,Fa zdravotníctva,štátna,2021,104,1,0,0,0
133,AKADÉMIA OZBROJ.SÍL,Vojenská fakulta AOS,štátna,2021,139,66,0,5,0


#### - všetky roky 2009-2021: načítanie dát o absolventoch, predspracovanie a uloženie do csv

In [32]:
data_paths = ['../data/cvtisr/vs2009/abvs2009.XLS', '../data/cvtisr/vs2010/abvs2010.XLS',
             '../data/cvtisr/vs2011/abvs2011.XLS', '../data/cvtisr/vs2012/abvs2012.XLS',
             '../data/cvtisr/vs2013/abvs2013.XLS', '../data/cvtisr/vs2014/abvs2014.xls',
             '../data/cvtisr/vs2015/abvs2015.xls', '../data/cvtisr/vs2016/abvs2016.xls',
             '../data/cvtisr/vs2017/abvs2017.xls', '../data/cvtisr/vs2018/abvs_2018.xls',
             '../data/cvtisr/vs2019/abvs_2019.xls', '../data/cvtisr/vs2020/abvs_2020.xls',
             '../data/cvtisr/vs2021/abvs_2021.xls']

In [33]:
all_graduates = pd.DataFrame()
for path in data_paths:
    year = path[-8:-4]
    print(f"Preprocessing data from year {year}")
    graduates_year = load_all_graduates(file_path=path, year=year)
    
    all_graduates = pd.concat([all_graduates, graduates_year]).reset_index(drop=True)
    
all_graduates.to_csv(f'../preprocessing/data_cvtisr_preprocessed/cvtisr_graduates.csv', sep=';', index=False)
    

Preprocessing data from year 2009
Preprocessing data from year 2010
Preprocessing data from year 2011
Preprocessing data from year 2012
Preprocessing data from year 2013
Preprocessing data from year 2014
Preprocessing data from year 2015
Preprocessing data from year 2016
Preprocessing data from year 2017
Preprocessing data from year 2018
Preprocessing data from year 2019
Preprocessing data from year 2020
Preprocessing data from year 2021
